In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [128]:
# parser v1
# parse records to list of dicts
# values mostly are strings
# easy to iterate through records
records = []
with open('Webscope_R6B/ydata-fp-td-clicks-v2_0.20111002') as f:
    for line in f.readlines():
        components = line.strip().split(' ')
        record = {}
        record['timestamp'] = components[0]
        record['displayed'] = components[1]
        record['clicked'] = int(components[2])
        # not using user feature
        record['user_features'] = []
        for i in range(4, len(components)):
            if components[i].startswith('|'):
                n = i
                break
            #record['user_features'].append(int(components[i]))
        record['pool'] = components[n:]    
        records.append(record)

In [129]:
records[0]

{'clicked': 0,
 'displayed': 'id-560620',
 'pool': ['|id-552077',
  '|id-555224',
  '|id-555528',
  '|id-559744',
  '|id-559855',
  '|id-560290',
  '|id-560518',
  '|id-560620',
  '|id-563115',
  '|id-563582',
  '|id-563643',
  '|id-563787',
  '|id-563846',
  '|id-563938',
  '|id-564335',
  '|id-564418',
  '|id-564604',
  '|id-565364',
  '|id-565479',
  '|id-565515',
  '|id-565533',
  '|id-565561',
  '|id-565589',
  '|id-565648',
  '|id-565747',
  '|id-565822'],
 'timestamp': '1317513291',
 'user_features': [1,
  9,
  11,
  13,
  23,
  16,
  18,
  17,
  19,
  15,
  43,
  14,
  39,
  30,
  66,
  50,
  27,
  104,
  20]}

In [152]:
len(records)

1607525

In [172]:
# offline simulation for AG_L
def AG_L(n=1000000, m=25, life_threshold=.3):
    # records should be long enough
    '''
    n: number of turns to play
    m: number of turns to initialize
    '''
    global records
    global lives # dict of articles to their lifespans
    rewards = 0
    i = 0
    X = {} # expectations
    t = 1
    
    # initialization
    while t<=m:
        displayed = records[i]['displayed']
        clicked = records[i]['clicked']
        if displayed not in X.keys():
            X[displayed] = np.array([clicked, 1])
            print('initialized',displayed,':',X[displayed],'t/i:',t,i)
            t += 1
        i += 1
    
    while t<=n:
        best = max(X.items(), key= lambda x:x[1][0]/x[1][1])
        X_max = best[1][0]/best[1][1]
        print(t, X_max)
        if np.random.binomial(1, X_max): # exploit
            while i < len(records): # look for target arm
                displayed = records[i]['displayed']
                clicked = records[i]['clicked']
                if displayed == best[0]:
                    rewards += clicked
                    print('exploited', displayed, ':', clicked, 't/i:', t,i)
                    break
                i += 1
                
        else: # explore
            while i < len(records):
                displayed = records[i]['displayed']
                clicked = records[i]['clicked']
                time = int(records[i]['timestamp'])
                born = filter(lambda x: x[1][0]<time, lives.items()) # fileter out unborn articles from lives
                life_rank = sorted(born, key= lambda x: x[1][1], reverse= True) # sort born articles by their expiring date
                young = life_rank[ : int(len(life_rank)*life_threshold)] # articles among top $life_threshold$% on the life ranking
                if '|'+displayed in list(zip(*young))[0]:
                    rewards += clicked
                    print('explored', displayed, ':', clicked, 't/i:', t,i)
                    break
                i += 1

        X[displayed] = X.get(displayed, np.array([0, 0])) + np.array([clicked, 1])
        t += 1
        i += 1   
    return rewards

In [174]:
AG_L(n=10000)

initialized id-560620 : [0 1] t/i: 1 0
initialized id-565648 : [0 1] t/i: 2 1
initialized id-563115 : [0 1] t/i: 3 2
initialized id-552077 : [0 1] t/i: 4 3
initialized id-564335 : [0 1] t/i: 5 4
initialized id-565589 : [0 1] t/i: 6 5
initialized id-563938 : [1 1] t/i: 7 7
initialized id-560290 : [0 1] t/i: 8 8
initialized id-563643 : [0 1] t/i: 9 9
initialized id-565822 : [0 1] t/i: 10 15
initialized id-563787 : [0 1] t/i: 11 16
initialized id-555528 : [0 1] t/i: 12 21
initialized id-565364 : [0 1] t/i: 13 26
initialized id-559855 : [0 1] t/i: 14 30
initialized id-560518 : [0 1] t/i: 15 31
initialized id-565561 : [0 1] t/i: 16 32
initialized id-564604 : [0 1] t/i: 17 33
initialized id-563846 : [0 1] t/i: 18 54
initialized id-563582 : [0 1] t/i: 19 55
initialized id-565533 : [0 1] t/i: 20 57
initialized id-565479 : [0 1] t/i: 21 60
initialized id-559744 : [0 1] t/i: 22 65
initialized id-565515 : [0 1] t/i: 23 67
initialized id-564418 : [0 1] t/i: 24 69
initialized id-565747 : [0 1] t/i:

explored id-564418 : 0 t/i: 167 1217
168 0.08333333333333333
explored id-564418 : 0 t/i: 168 1222
169 0.08333333333333333
explored id-563846 : 1 t/i: 169 1224
170 0.08333333333333333
explored id-565822 : 0 t/i: 170 1227
171 0.08
explored id-564418 : 0 t/i: 171 1229
172 0.08
explored id-563846 : 0 t/i: 172 1231
173 0.08
explored id-565515 : 0 t/i: 173 1232
174 0.08
explored id-565515 : 0 t/i: 174 1233
175 0.08
explored id-563846 : 0 t/i: 175 1235
176 0.08
explored id-565515 : 1 t/i: 176 1237
177 0.08695652173913043
explored id-564418 : 0 t/i: 177 1240
178 0.08695652173913043
explored id-555528 : 0 t/i: 178 1244
179 0.08695652173913043
explored id-564418 : 0 t/i: 179 1246
180 0.08695652173913043
explored id-555224 : 0 t/i: 180 1253
181 0.08695652173913043
explored id-555528 : 0 t/i: 181 1257
182 0.08695652173913043
explored id-564418 : 0 t/i: 182 1260
183 0.08695652173913043
explored id-565515 : 0 t/i: 183 1261
184 0.08333333333333333
exploited id-565515 : 0 t/i: 184 1273
185 0.08
explor

explored id-555528 : 0 t/i: 410 2730
411 0.1
explored id-555224 : 0 t/i: 411 2732
412 0.1
explored id-565822 : 0 t/i: 412 2735
413 0.09859154929577464
exploited id-565822 : 0 t/i: 413 2753
414 0.09722222222222222
explored id-564418 : 0 t/i: 414 2754
415 0.09722222222222222
explored id-563846 : 0 t/i: 415 2756
416 0.09722222222222222
explored id-555224 : 0 t/i: 416 2757
417 0.09722222222222222
exploited id-565822 : 0 t/i: 417 2784
418 0.0958904109589041
explored id-555224 : 0 t/i: 418 2790
419 0.0958904109589041
explored id-565515 : 1 t/i: 419 2799
420 0.0958904109589041
exploited id-565822 : 0 t/i: 420 2876
421 0.0945945945945946
explored id-555528 : 0 t/i: 421 2878
422 0.0945945945945946
explored id-565515 : 0 t/i: 422 2879
423 0.0945945945945946
explored id-565515 : 0 t/i: 423 2882
424 0.0945945945945946
explored id-565822 : 0 t/i: 424 2885
425 0.09333333333333334
exploited id-565822 : 0 t/i: 425 2886
426 0.09210526315789473
exploited id-565822 : 0 t/i: 426 2930
427 0.090909090909090

explored id-563846 : 0 t/i: 616 4068
617 0.11538461538461539
explored id-565822 : 0 t/i: 617 4069
618 0.11538461538461539
exploited id-565515 : 0 t/i: 618 4073
619 0.11428571428571428
explored id-555528 : 0 t/i: 619 4085
620 0.11428571428571428
explored id-564418 : 0 t/i: 620 4089
621 0.11428571428571428
explored id-555224 : 0 t/i: 621 4095
622 0.11428571428571428
explored id-563846 : 0 t/i: 622 4098
623 0.11428571428571428
explored id-563846 : 0 t/i: 623 4099
624 0.11428571428571428
explored id-565515 : 0 t/i: 624 4102
625 0.11320754716981132
explored id-565515 : 0 t/i: 625 4105
626 0.11214953271028037
explored id-555528 : 1 t/i: 626 4112
627 0.11214953271028037
explored id-555224 : 0 t/i: 627 4115
628 0.11214953271028037
explored id-563846 : 0 t/i: 628 4121
629 0.11214953271028037
explored id-555528 : 0 t/i: 629 4123
630 0.11214953271028037
explored id-563643 : 0 t/i: 630 4131
631 0.11214953271028037
exploited id-565515 : 0 t/i: 631 4159
632 0.1111111111111111
explored id-565515 : 0 

explored id-565822 : 0 t/i: 822 5326
823 0.1069182389937107
exploited id-565515 : 0 t/i: 823 5367
824 0.10625
explored id-563643 : 0 t/i: 824 5376
825 0.10625
explored id-565822 : 0 t/i: 825 5377
826 0.10625
explored id-555224 : 1 t/i: 826 5380
827 0.10625
explored id-563846 : 0 t/i: 827 5388
828 0.10625
explored id-563846 : 0 t/i: 828 5391
829 0.10625
explored id-563643 : 0 t/i: 829 5393
830 0.10625
explored id-563846 : 1 t/i: 830 5395
831 0.10625
explored id-565822 : 0 t/i: 831 5397
832 0.10625
explored id-565822 : 0 t/i: 832 5399
833 0.10625
explored id-564418 : 0 t/i: 833 5401
834 0.10625
explored id-564418 : 0 t/i: 834 5409
835 0.10625
explored id-555528 : 0 t/i: 835 5415
836 0.10625
explored id-565515 : 0 t/i: 836 5419
837 0.10559006211180125
exploited id-565515 : 1 t/i: 837 5473
838 0.1111111111111111
explored id-555528 : 0 t/i: 838 5474
839 0.1111111111111111
explored id-565822 : 0 t/i: 839 5477
840 0.1111111111111111
explored id-563846 : 0 t/i: 840 5483
841 0.1111111111111111


...

explored id-555224 : 0 t/i: 9282 49730
9283 0.06306306306306306
explored id-563643 : 0 t/i: 9283 49733
9284 0.06306306306306306
explored id-565822 : 0 t/i: 9284 49735
9285 0.06302250803858521
explored id-565515 : 0 t/i: 9285 49736
9286 0.06302250803858521
explored id-555528 : 0 t/i: 9286 49743
9287 0.06302250803858521
explored id-565822 : 0 t/i: 9287 49745
9288 0.06298200514138817
explored id-563643 : 0 t/i: 9288 49747
9289 0.06298200514138817
exploited id-565822 : 0 t/i: 9289 49805
9290 0.06294155427103404
explored id-564418 : 0 t/i: 9290 49807
9291 0.06294155427103404
exploited id-565822 : 0 t/i: 9291 49812
9292 0.06290115532734275
explored id-564418 : 0 t/i: 9292 49818
9293 0.06290115532734275
explored id-555528 : 0 t/i: 9293 49822
9294 0.06290115532734275
explored id-563846 : 0 t/i: 9294 49824
9295 0.06290115532734275
explored id-565515 : 0 t/i: 9295 49833
9296 0.06290115532734275
explored id-563643 : 0 t/i: 9296 49837
9297 0.06290115532734275
explored id-565515 : 1 t/i: 9297 49840

9510 0.06261740763932373
explored id-565822 : 0 t/i: 9510 50577
9511 0.0625782227784731
explored id-563643 : 0 t/i: 9511 50582
9512 0.0625782227784731
explored id-555224 : 0 t/i: 9512 50593
9513 0.0625782227784731
explored id-563643 : 0 t/i: 9513 50595
9514 0.0625782227784731
explored id-563643 : 0 t/i: 9514 50596
9515 0.0625782227784731
explored id-565822 : 0 t/i: 9515 50601
9516 0.06253908692933083
explored id-563643 : 0 t/i: 9516 50602
9517 0.06253908692933083
explored id-563846 : 0 t/i: 9517 50607
9518 0.06253908692933083
explored id-565822 : 0 t/i: 9518 50608
9519 0.0625
explored id-563643 : 0 t/i: 9519 50610
9520 0.0625
explored id-555224 : 0 t/i: 9520 50612
9521 0.0625
explored id-563643 : 0 t/i: 9521 50613
9522 0.0625
explored id-563643 : 0 t/i: 9522 50614
9523 0.0625
explored id-563643 : 0 t/i: 9523 50615
9524 0.0625
explored id-565822 : 0 t/i: 9524 50619
9525 0.06246096189881324
explored id-565822 : 0 t/i: 9525 50629
9526 0.062421972534332085
exploited id-565822 : 0 t/i: 9526

exploited id-565822 : 0 t/i: 9709 51430
9710 0.06315144083384426
explored id-564418 : 0 t/i: 9710 51432
9711 0.06315144083384426
explored id-555528 : 0 t/i: 9711 51442
9712 0.06315144083384426
explored id-555224 : 0 t/i: 9712 51445
9713 0.06315144083384426
explored id-555528 : 0 t/i: 9713 51447
9714 0.06315144083384426
explored id-563643 : 0 t/i: 9714 51448
9715 0.06315144083384426
explored id-565822 : 0 t/i: 9715 51451
9716 0.06311274509803921
explored id-555528 : 0 t/i: 9716 51456
9717 0.06311274509803921
explored id-564418 : 0 t/i: 9717 51458
9718 0.06311274509803921
explored id-563846 : 0 t/i: 9718 51461
9719 0.06311274509803921
explored id-563846 : 0 t/i: 9719 51472
9720 0.06311274509803921
explored id-565822 : 0 t/i: 9720 51474
9721 0.06307409675443969
explored id-563643 : 0 t/i: 9721 51475
9722 0.06307409675443969
explored id-555224 : 0 t/i: 9722 51483
9723 0.06307409675443969
explored id-564418 : 0 t/i: 9723 51484
9724 0.06307409675443969
explored id-565515 : 0 t/i: 9724 51490


explored id-564418 : 0 t/i: 9933 52337
9934 0.06354916067146282
explored id-565515 : 0 t/i: 9934 52339
9935 0.06354916067146282
explored id-564418 : 0 t/i: 9935 52342
9936 0.06354916067146282
explored id-564418 : 0 t/i: 9936 52344
9937 0.06354916067146282
explored id-564418 : 0 t/i: 9937 52349
9938 0.06354916067146282
explored id-565515 : 0 t/i: 9938 52350
9939 0.06354916067146282
explored id-565822 : 0 t/i: 9939 52351
9940 0.06351108448172559
explored id-563643 : 0 t/i: 9940 52352
9941 0.06351108448172559
explored id-565822 : 0 t/i: 9941 52353
9942 0.06347305389221557
explored id-565515 : 0 t/i: 9942 52360
9943 0.06347305389221557
explored id-555528 : 0 t/i: 9943 52364
9944 0.06347305389221557
explored id-564418 : 0 t/i: 9944 52370
9945 0.06347305389221557
explored id-565822 : 0 t/i: 9945 52372
9946 0.06343506882106523
explored id-564418 : 0 t/i: 9946 52373
9947 0.06343506882106523
explored id-564418 : 0 t/i: 9947 52381
9948 0.06343506882106523
explored id-564418 : 0 t/i: 9948 52386
9

442

In [2]:
# parser v2
# parse records to dict of lists
# values are mostly strings
# easy to feed in pandas
records = {'timestamp':[],
          'displayed':[],
          'clicked':[],
          'user_features':[],
          'pool':[]}
with open('test.txt') as f:
    for line in f.readlines():
        components = line.strip().split(' ')
        records['timestamp'].append(components[0])
        records['displayed'].append(components[1])
        records['clicked'].append(components[2])
        user_features = []
        for i in range(4, len(components)):
            if components[i].startswith('|'):
                n = i
                break
            user_features.append(int(components[i]))
        records['user_features'].append(user_features)
        records['pool'].append(components[n:])

In [3]:
df = pd.DataFrame(records)
df.head()

,timestamp,displayed,clicked,user_features,pool
0,1317513291,id-560620,0,"[1, 9, 11, 13, 23, 16, 18, 17, 19, 15, 43, 14,...","[|id-552077, |id-555224, |id-555528, |id-55974..."
1,1317513291,id-565648,0,"[1, 9, 11, 13, 16, 15, 14]","[|id-552077, |id-555224, |id-555528, |id-55974..."
2,1317513291,id-563115,0,"[1, 8, 11, 36, 13, 22, 23, 16, 18, 54, 24, 26,...","[|id-552077, |id-555224, |id-555528, |id-55974..."
3,1317513292,id-552077,0,"[1, 7, 11, 37, 13, 23, 16, 18, 17, 35, 15, 14,...","[|id-552077, |id-555224, |id-555528, |id-55974..."
4,1317513292,id-564335,0,[1],"[|id-552077, |id-555224, |id-555528, |id-55974..."


In [7]:
df[df['clicked']=='1']

,timestamp,displayed,clicked,user_features,pool
7,1317513293,id-563938,1,"[1, 13, 23, 33, 18, 24, 17, 19, 29, 14]","[|id-552077, |id-555224, |id-555528, |id-55974..."
13,1317513293,id-564335,1,"[1, 7, 11, 36, 65, 13, 22, 23, 33, 32, 16, 18,...","[|id-552077, |id-555224, |id-555528, |id-55974..."
39,1317513295,id-564335,1,"[1, 8, 12, 36, 65, 13, 22, 23, 16, 18, 24, 26,...","[|id-552077, |id-555224, |id-555528, |id-55974..."
144,1317513299,id-565747,1,"[1, 7, 12, 13, 22, 23, 33, 32, 58, 16, 18, 24,...","[|id-552077, |id-555224, |id-555528, |id-55974..."


In [8]:
df.shape

(148, 5)

In [89]:
# parser that watches article lifespans
lives = {}
with open('Webscope_R6B/ydata-fp-td-clicks-v2_0.20111002') as f:
    for line in f.readlines():
        components = line.strip().split(' ')
        for component in components:
            if component.startswith('|id-'):
                timestamp = int(components[0])
                if component in lives.keys():
                    lives[component][1] = timestamp
                else:
                    lives[component] = [timestamp, timestamp]
                '''if component in lives:
                    lives[component].add(int(components[0]))
                else:
                    lives[component] = set()'''

In [22]:
lives.keys()

dict_keys(['|id-552077', '|id-555224', '|id-555528', '|id-559744', '|id-559855', '|id-560290', '|id-560518', '|id-560620', '|id-563115', '|id-563582', '|id-563643', '|id-563787', '|id-563846', '|id-563938', '|id-564335', '|id-564418', '|id-564604', '|id-565364', '|id-565479', '|id-565515', '|id-565533', '|id-565561', '|id-565589', '|id-565648', '|id-565747', '|id-565822', '|id-565930', '|id-566013', '|id-566022', '|id-566092', '|id-560805', '|id-564371', '|id-562265', '|id-565980', '|id-566431', '|id-566439', '|id-559833', '|id-566541', '|id-562374', '|id-566587', '|id-566478', '|id-566573', '|id-566602', '|id-562637', '|id-566631', '|id-566689', '|id-566726', '|id-566825', '|id-566838'])

In [125]:
for v in lives.values():
    print(v)

[1317513291, 1317515399]
[1317513291, 1317588299]
[1317513291, 1317587099]
[1317513291, 1317515699]
[1317513291, 1317587099]
[1317513291, 1317587099]
[1317513291, 1317578399]
[1317513291, 1317521099]
[1317513291, 1317581999]
[1317513291, 1317583799]
[1317513291, 1317599694]
[1317513291, 1317515699]
[1317513291, 1317599694]
[1317513291, 1317519299]
[1317513291, 1317571199]
[1317513291, 1317599694]
[1317513291, 1317520799]
[1317513291, 1317535499]
[1317513291, 1317583499]
[1317513291, 1317587399]
[1317513291, 1317583499]
[1317513291, 1317549899]
[1317513291, 1317577499]
[1317513291, 1317586799]
[1317513291, 1317570299]
[1317513291, 1317599694]
[1317517500, 1317582299]
[1317525000, 1317585899]
[1317527100, 1317594299]
[1317529500, 1317564299]
[1317565500, 1317599694]
[1317568200, 1317599694]
[1317569100, 1317599694]
[1317570600, 1317574799]
[1317572400, 1317599694]
[1317573300, 1317599694]
[1317577200, 1317599694]
[1317577800, 1317599694]
[1317578100, 1317599694]
[1317579300, 1317599694]


In [161]:
len(filter(lambda x: x[1][0]<timestamp, lives.items()))

TypeError: object of type 'filter' has no len()

In [155]:
timestamp = 1317513600
life_rank = sorted(filter(lambda x: x[1][0]<timestamp, lives.items()), key= lambda x: x[1][1], reverse= True)
life_rank

[('|id-563643', [1317513291, 1317599694]),
 ('|id-563846', [1317513291, 1317599694]),
 ('|id-564418', [1317513291, 1317599694]),
 ('|id-565822', [1317513291, 1317599694]),
 ('|id-555224', [1317513291, 1317588299]),
 ('|id-565515', [1317513291, 1317587399]),
 ('|id-555528', [1317513291, 1317587099]),
 ('|id-559855', [1317513291, 1317587099]),
 ('|id-560290', [1317513291, 1317587099]),
 ('|id-565648', [1317513291, 1317586799]),
 ('|id-563582', [1317513291, 1317583799]),
 ('|id-565479', [1317513291, 1317583499]),
 ('|id-565533', [1317513291, 1317583499]),
 ('|id-563115', [1317513291, 1317581999]),
 ('|id-560518', [1317513291, 1317578399]),
 ('|id-565589', [1317513291, 1317577499]),
 ('|id-564335', [1317513291, 1317571199]),
 ('|id-565747', [1317513291, 1317570299]),
 ('|id-565561', [1317513291, 1317549899]),
 ('|id-565364', [1317513291, 1317535499]),
 ('|id-560620', [1317513291, 1317521099]),
 ('|id-564604', [1317513291, 1317520799]),
 ('|id-563938', [1317513291, 1317519299]),
 ('|id-5597

In [168]:
life_rank[:int(len(life_rank)*.3)]

[('|id-563643', [1317513291, 1317599694]),
 ('|id-563846', [1317513291, 1317599694]),
 ('|id-564418', [1317513291, 1317599694]),
 ('|id-565822', [1317513291, 1317599694]),
 ('|id-555224', [1317513291, 1317588299]),
 ('|id-565515', [1317513291, 1317587399]),
 ('|id-555528', [1317513291, 1317587099])]

In [99]:
int(len(life_rank)*.3)

6

In [169]:
list(zip(*life_rank))[0]

('|id-563643',
 '|id-563846',
 '|id-564418',
 '|id-565822',
 '|id-555224',
 '|id-565515',
 '|id-555528',
 '|id-559855',
 '|id-560290',
 '|id-565648',
 '|id-563582',
 '|id-565479',
 '|id-565533',
 '|id-563115',
 '|id-560518',
 '|id-565589',
 '|id-564335',
 '|id-565747',
 '|id-565561',
 '|id-565364',
 '|id-560620',
 '|id-564604',
 '|id-563938',
 '|id-559744',
 '|id-563787',
 '|id-552077')